In [1]:
import torch
from torch import nn
from torch.nn import functional as F 

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0546,  0.0242,  0.0609,  0.1338, -0.1225, -0.0509,  0.2274,  0.1015,
         -0.0883,  0.0474],
        [-0.0384, -0.0720,  0.1517,  0.1071, -0.1906,  0.0570,  0.1647,  0.2128,
         -0.0450,  0.1758]], grad_fn=<AddmmBackward>)

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[ 0.1086, -0.4847,  0.0814,  0.1358, -0.0889,  0.2450, -0.1848, -0.3644,
          0.0557, -0.0858],
        [ 0.0445, -0.2519, -0.0391,  0.1343, -0.1033,  0.3389, -0.2066, -0.2878,
          0.1330,  0.1034]], grad_fn=<AddmmBackward>)

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
        print(self._modules)

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

OrderedDict([(Linear(in_features=20, out_features=256, bias=True), Linear(in_features=20, out_features=256, bias=True)), (ReLU(), ReLU()), (Linear(in_features=256, out_features=10, bias=True), Linear(in_features=256, out_features=10, bias=True))])


tensor([[-0.0297,  0.0272, -0.1285, -0.2007, -0.0646, -0.2681, -0.2614, -0.0663,
         -0.3634,  0.0088],
        [-0.0883,  0.0200, -0.1204, -0.2466,  0.0664, -0.1282, -0.2293, -0.1819,
         -0.2788,  0.0734]], grad_fn=<AddmmBackward>)

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=True)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight)+1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [7]:
net = FixedHiddenMLP()
net(X)

tensor(-0.0906, grad_fn=<SumBackward0>)

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

In [9]:
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1883, grad_fn=<SumBackward0>)